# HW4 : Math for Robotics

Author: Ruffin White  
Course: CSE291  
Date: Mar 9 2018

In [ ]:
# Make plots inline
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('pdf', 'svg')
# set_matplotlib_formats('png', 'pdf')
set_matplotlib_formats('png')

# import modules for plotting and data analysis
import matplotlib.pyplot as plt

## 1. 

> Use the ATT dataset http://www.cl.cam.ac.uk/research/dtg/attarchive/pub/data/att faces.zip. to compute subspaces for the PCA and LDA methods. Provide illustration of the respective 1st, 2nd and 3rd eigenvectors. Compute the recognition rates for the test-set. Report

* Correct classification
* Incorrect classification

> Provide at least one suggestion for how you might improve performance of the system

In [ ]:
import numpy as np
import pandas as pd
import pathlib

from collections import OrderedDict
from PIL import Image

In [ ]:
def read_data(data_dir_path):
    rows = []
    height, width = 112, 92
    flattened = height * width # 10304
    X_keys = ["X_" + str(i) for i in range(flattened)]
    paths = pathlib.Path(data_dir_path).glob('**/*.pgm')
    for path in paths:
        im = Image.open(str(path))
        X_values = np.array(im).flatten()
        y_value = path.parent.name
        row = OrderedDict(zip(X_keys, X_values))
        row['y'] = y_value
        rows.append(row)

    df = pd.DataFrame(rows)
    return df

In [ ]:
data_dir_path = 'data/att_faces/'

In [ ]:
df = read_data(data_dir_path)
X = df.drop('y',axis=1)
y = df['y']

In [ ]:
plt.figure(figsize=(12,10))
numbers = [0,1,398,399]
for i in range(4):
    im = X.loc[numbers[i]].values.reshape(112, 92)
    label = y.loc[numbers[i]]
    plt.subplot(2, 2, i+1)
    plt.imshow(im, cmap='gray')
    plt.axis("off")
    plt.title(label)
plt.tight_layout()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

import math
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(),
                                                    test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=X_train.shape[1])
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
print('X_std_pca.shape:', X_train_pca.shape)
print('pca.components_:', pca.components_.shape)

In [ ]:
plt.figure(figsize=(12,10))
for i in range(20):
    eigenvector = pca.components_[i].reshape(112, 92)
    plt.subplot(4, 5, i+1)
    plt.imshow(eigenvector, cmap='gray')
    plt.title("Eigenvector: {0}".format(i+1))
    plt.axis("off")
plt.tight_layout()

## 2.

> In robotics sound source localization is a frequent challenge. In the file http://www.hichristensen.com/demo-delay.wav estimate the delay between the two sound channels using FFT. provide an explanation of how you computes the delay between the two channels.

> For both questions provide a description of the approach adopted, the associated code and a description of your results.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

from scipy import signal
from scipy.fftpack import fft
from scipy.fftpack import ifft
from scipy.io import wavfile

In [ ]:
fs, data = wavfile.read('data/demo-delay.wav')
L = data.T[0]
R = data.T[1]

In [ ]:
plt.plot(np.array_split(L, 100)[0])
plt.plot(np.array_split(R, 100)[0])
plt.show()

In [ ]:
nsamples = L.size

# regularize datasets by subtracting mean and dividing by s.d.
L = L - L.mean()
L = L/L.std()
R = R - R.mean()
R = R/R.std()

# Find cross-correlation
xcorr = signal.correlate(L, R)

# delta time array to match xcorr
dt = np.arange(1-nsamples, nsamples)

recovered_time_shift = dt[xcorr.argmax()]

In [ ]:
hz = 44.1e3
p = 1/hz
sec = recovered_time_shift * p

print("Recovered sample shift: ", recovered_time_shift)
print("Recovered time shift (sec): ", sec)

In [ ]:
ax = plt.subplot(111)
ax.plot(xcorr)
# ax.set_yscale("log", nonposy='clip')
plt.show()

In [ ]:
ax = plt.subplot(111)
ax.plot(xcorr[nsamples-1000:nsamples+1000])
# ax.set_yscale("log", nonposy='clip')
plt.show()

In [ ]:
plt.plot(np.array_split(L[0:], 1000)[0][100:150])
plt.plot(np.array_split(R[21:], 1000)[0][100:150])
plt.plot(np.array_split(R[23:], 1000)[0][100:150])
plt.plot(np.array_split(R[22:], 1000)[0][100:150],'--')
plt.show()

In [ ]:
343*sec

In [ ]:
L_fft = fft(np.array_split(L, 10)[0])
R_fft = fft(np.flip(np.array_split(R, 10)[0], 0))

In [ ]:
s1 = np.array(L.shape)
s2 = np.array(R.shape)

shape = s1 + s2 - 1
fshape = [scipy.fftpack.helper.next_fast_len(int(d)) for d in shape]
fslice = tuple([slice(0, int(sz)) for sz in shape])

sp1 = np.fft.rfftn(L, fshape)
sp2 = np.fft.rfftn(np.flip(R, axis=0), fshape)
ret = (np.fft.irfftn(sp1 * sp2, fshape)[fslice].copy())

In [ ]:
ax = plt.subplot(111)
ax.plot(ret,'r')
# ax.set_xscale("log")
# ax.set_yscale("log")
plt.show()

In [ ]:
ret_maxi = np.abs(ret).argmax() 
ret_max = ret[ret_maxi]

In [ ]:
ret_maxi - ret.size/2

In [ ]:
autocorr = signal.fftconvolve(L, np.flip(R, axis=0), mode='full')

In [ ]:
ax = plt.subplot(111)
ax.plot(autocorr,'r')
# ax.set_xscale("log")
# ax.set_yscale("log")
plt.show()

In [ ]:
autocorr.argmax() - autocorr.size/2

In [ ]:
ax = plt.subplot(111)
ax.plot(autocorr[int(autocorr.size/2)-1000:int(autocorr.size/2)+1000], 'r')
# ax.set_yscale("log", nonposy='clip')
plt.show()